## (1) Load Processed Data in the Paper

Currently, we support norman / adamson / dixit.

In [2]:
import os
os.chdir("/gpfs/home/asun/jin_lab/get/GEARS/demo/")
print(os.getcwd())

import sys
sys.path.append(os.path.abspath('..'))

from gears import PertData

/gpfs/group/jin/asun/get/GEARS/demo


In [2]:
import sys
sys.path.append(os.path.abspath('..'))

from gears import PertData

pert_data = PertData('./data') # specific saved folder
pert_data.load(data_name = 'norman') # specific dataset name
pert_data.prepare_split(split = 'simulation', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
Found local copy...
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['RHOXF2BB+ctrl' 'LYL1+IER5L' 'ctrl+IER5L' 'KIAA1804+ctrl' 'IER5L+ctrl'
 'RHOXF2BB+ZBTB25' 'RHOXF2BB+SET']
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:9
combo_seen1:43
combo_seen2:19
unseen_single:36
Done!
Creating dataloaders....
Done!


here1


## (2) Create your own Perturb-Seq data
Prepare a scanpy adata object with 
1. `adata.obs` dataframe has `condition` and `cell_type` columns, where `condition` is the perturbation name for each cell. Control cells have condition format of `ctrl`, single perturbation has condition format of `A+ctrl` or `ctrl+A`, combination perturbation has condition format of `A+B`.
2. `adata.var` dataframe has `gene_name` column, where each gene name is the gene symbol.
3. `adata.X` stores the post-perturbed gene expression. 

Here an example using dixit 2016 dataset.

In [3]:
adata_sim = sc.read_h5ad('simulated_adata.h5ad')
adata_sim

AnnData object with n_obs × n_vars = 25738 × 5000
    obs: 'condition', 'perturbation'
    var: 'variable_gene', 'gene_name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'

In [7]:
adata_sim.obs = adata_sim.obs.rename(columns={"condition": "target_gene"})


In [13]:
adata_sim.obs['target_gene'] = adata_sim.obs['target_gene'].replace('ctrl', 'non-targeting')


In [14]:
adata_sim.write("/gpfs/home/asun/jin_lab/get/cell-eval/runs/simulated_adata.h5ad")

In [9]:
adata_real = sc.read_h5ad('/gpfs/home/asun/jin_lab/get/GEARS/demo/data/boli/perturb_processed.h5ad')
adata_real

AnnData object with n_obs × n_vars = 35527 × 5000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'seurat_clusters', 'Assign', 'scds', 'cxds', 'bcds', 'Sample', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'CT', 'mito', 'BioSamp', 'CT2', 'ForPlot', 'Remove', 'active_ident', 'Assign_clean', 'condition', 'cell_type', 'dose_val', 'control', 'condition_name'
    var: 'variable_gene', 'gene_name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p', 'non_dropout_gene_idx', 'non_zeros_gene_idx', 'rank_genes_groups_cov_all', 'top_non_dropout_de_20', 'top_non_zero_de_20'

In [ ]:
adata_real.obs = adata_real.obs.rename(columns={"condition": "target_gene"})


In [12]:
exclude_genes = ['TRIO+ctrl', 'DAB1+ctrl', 'CX3CL1+ctrl']
# Filter out those rows
adata_real_filtered = adata_real[~adata_real.obs['target_gene'].isin(exclude_genes)].copy()
adata_real_filtered.obs['target_gene'] = adata_real_filtered.obs['target_gene'].replace('ctrl', 'non-targeting')
adata_real_filtered.write("/gpfs/home/asun/jin_lab/get/cell-eval/runs/real_adata.h5ad")

In [4]:
gene_list = adata_sim.var_names.to_numpy()

In [5]:
gene_list

array(['4', '5', '7', ..., '32205', '32207', '32261'],
      shape=(5000,), dtype=object)

In [6]:
import pandas as pd

pd.Series(gene_list).to_csv("gene_list.csv", index=False, header=False)

In [1]:
import scanpy as sc
adata = sc.read_h5ad('/gpfs/home/asun/jin_lab/get/raw_data/ctxobj.h5ad')
adata

AnnData object with n_obs × n_vars = 11471 × 32285
    obs: 'Name', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'scds', 'cxds', 'bcds', 'RNA_snn_res.0.8', 'seurat_clusters', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'feature_call', 'num_features', 'num_umis', 'nCount_Crispr', 'nFeature_Crispr', 'RNA_snn_res.1.5', 'lowQC', 'CellType', 'Perc', 'CBC', 'antisense', 'intergenic', 'intronic', 'exonic', 'multi', 'unmapped', 'highConf', 'polyA', 'TSO', 'spliced', 'percent_qual_cbc', 'percent_qual_umi', 'nUMI', 'UTR', 'total', 'Rem', 'CellType_Gen', 'Mito', 'demux_type', 'assignment', 'Keep', 'Max', 'MaxVal', 'SecMaxVal', 'demuxHC', 'demuxCR', 'demuxDS', 'assignDS', 'pertgene', 'active_ident'
    var: 'variable_gene'
    uns: 'RNA_snn_res.0.8_colors', 'active_ident_colors', 'seurat_clusters_colors'
    obsm: 'X_gbm', 'X_pca', 'X_umap'
    layers: 'raw_count'

In [6]:
adata.var.head(2)

,variable_gene,gene_name
0,True,Xkr4
1,False,Gm1992


In [4]:
# Store the current index (gene names) into a new column called 'gene_name'
adata.var['gene_name'] = adata.var.index

# Optionally reset the index if you want to replace it with default integers
adata.var.reset_index(drop=True, inplace=True)

In [7]:
adata.obs['pertgene'].value_counts()

pertgene
ctrl                     2770
Tcf4                     2276
Foxg1                    1982
Nr2f1                    1885
Tbr1                     1448
                         ... 
Tcf4_2,Tbr1_3,Tbr1_1        1
Tcf4_3,Tcf4_1,Foxg1_1       1
Foxg1_1,GFP,Tcf4_2          1
Tcf4_3,Tcf4_2,Tbr1_1        1
Foxg1_1,GFP,Tcf4_1          1
Name: count, Length: 283, dtype: int64

In [8]:
adata = adata[~adata.obs['pertgene'].str.contains(',')]

In [9]:
adata.obs['pertgene'].value_counts()

pertgene
ctrl     2770
Tcf4     2276
Foxg1    1982
Nr2f1    1885
Tbr1     1448
Name: count, dtype: int64

In [10]:
adata.obs['condition'] = adata.obs['pertgene'].apply(
    lambda x: x if x == 'ctrl' else f"{x.split('+')[0].upper()}+ctrl"
)

In [11]:
adata.obs['condition'].value_counts()

condition
ctrl          2770
TCF4+ctrl     2276
FOXG1+ctrl    1982
NR2F1+ctrl    1885
TBR1+ctrl     1448
Name: count, dtype: int64

In [12]:
adata.obs['CellType'].value_counts()

CellType
Excit_L6CT_CTX      2918
Excit_L5IT          1103
CR                  1029
Excit_Upper          983
Excit_L6IT           948
Excit_L5NP_CTX       899
Excit_L2 IT ENTl     498
Excit_L5 PT CTX      470
Inhib_Lhx6+Sst-      435
Inhib_Sst            387
Excit_L6b/CT ENT     247
Excit_L6b CTX        226
Excit_Car3           131
Inhib_Id2             87
Name: count, dtype: int64

In [13]:
#adata.obs.rename(columns={'CellType': 'cell_type'}, inplace=True)

In [14]:
adata.obs['cell_type'].value_counts()

cell_type
Excit_L6CT_CTX      2918
Excit_L5IT          1103
CR                  1029
Excit_Upper          983
Excit_L6IT           948
Excit_L5NP_CTX       899
Excit_L2 IT ENTl     498
Excit_L5 PT CTX      470
Inhib_Lhx6+Sst-      435
Inhib_Sst            387
Excit_L6b/CT ENT     247
Excit_L6b CTX        226
Excit_Car3           131
Inhib_Id2             87
Name: count, dtype: int64

In [15]:
# test to see if the cell type split is the issue
adata.obs['cell_type'] = 'Neuron'

In [22]:
adata.obs.head(2)

,orig.ident,nCount_RNA,nFeature_RNA,seurat_clusters,Assign,scds,cxds,bcds,Sample,nCount_refAssay,...,CT,mito,BioSamp,CT2,ForPlot,Remove,active_ident,Assign_clean,condition,cell_type
batch1_samp1_chan1_AAACCTGAGACAAAGG-1,batch1,6831,2953,7,Bcl11b_0,1.323674,28996.749458,0.880423,batch1_samp1_chan1,6831,...,L6 CT CTX,0.029278,batch1_samp1,030 L6 CT CTX Glut,batch1_samp1_Bcl11b_0,False,7,Bcl11b,BCL11B+ctrl,Neuron
batch1_samp1_chan1_AAACCTGAGGCCGAAT-1,batch1,6016,2767,21,Bcl11b_0,0.487555,9829.195327,0.337222,batch1_samp1_chan1,6016,...,L6b CTX,0.016622,batch1_samp1,029 L6b CTX Glut,batch1_samp1_Bcl11b_0,False,21,Bcl11b,BCL11B+ctrl,Neuron


### Suggested normalization

For raw count data we recommend the following normalization and subsetting to the top 5000 most variable genes

In [16]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata,n_top_genes=5000, subset=True)

### Create dataloader

GEARS will take it from here. The new data processing takes around 15 minutes for 5K genes and 100K cells. 

In [18]:
import sys
sys.path.append('../')

from gears import PertData

pert_data = PertData('./data') # specific saved folder
pert_data.new_data_process(dataset_name = 'zheng', adata = adata) # specific dataset name and adata object
pert_data.load(data_path = './data/zheng') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split = 'simulation_single', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
Found local copy...
Creating pyg object for each cell in the data...
Creating dataset file...
100%|████████████████████████████████████████████████████████████████████████| 5/5 [01:01<00:00, 12.27s/it]
Done!
Saving new dataset pyg object at ./data/zheng/data_pyg/cell_graphs.pkl
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
[]
Local copy of pyg dataset is detected. Loading...
Done!
Creating new splits....
Saving new splits at ./data/zheng/splits/zheng_simulation_single_1_0.75.pkl
Done!
Creating dataloaders....
Done!


simulation_single
split_data returns: (AnnData object with n_obs × n_vars = 10361 × 5000
    obs: 'Name', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'scds', 'cxds', 'bcds', 'RNA_snn_res.0.8', 'seurat_clusters', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'feature_call', 'num_features', 'num_umis', 'nCount_Crispr', 'nFeature_Crispr', 'RNA_snn_res.1.5', 'lowQC', 'cell_type', 'Perc', 'CBC', 'antisense', 'intergenic', 'intronic', 'exonic', 'multi', 'unmapped', 'highConf', 'polyA', 'TSO', 'spliced', 'percent_qual_cbc', 'percent_qual_umi', 'nUMI', 'UTR', 'total', 'Rem', 'CellType_Gen', 'Mito', 'demux_type', 'assignment', 'Keep', 'Max', 'MaxVal', 'SecMaxVal', 'demuxHC', 'demuxCR', 'demuxDS', 'assignDS', 'pertgene', 'active_ident', 'condition', 'dose_val', 'control', 'condition_name', 'split'
    var: 'variable_gene', 'gene_name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'RNA_snn_res.0.8_colors', 'active_ident_colors'

# processing boli

In [9]:
import scanpy as sc
adata = sc.read_h5ad('/gpfs/home/asun/jin_lab/get/raw_data/seur.cur.h5ad')
adata

AnnData object with n_obs × n_vars = 35527 × 32285
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'seurat_clusters', 'Assign', 'scds', 'cxds', 'bcds', 'Sample', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'CT', 'mito', 'BioSamp', 'CT2', 'ForPlot', 'Remove', 'active_ident'
    var: 'variable_gene'

In [21]:
adata.var.head(2)

,variable_gene,gene_name
0,False,Xkr4
1,False,Gm1992


In [12]:
# Store the current index (gene names) into a new column called 'gene_name'
adata.var['gene_name'] = adata.var.index

# Optionally reset the index if you want to replace it with default integers
adata.var.reset_index(drop=True, inplace=True)

In [14]:
# Show the unique cell types
adata.obs['Assign'].value_counts()

Assign
Bcl11b_0    5367
Trio_0      4274
Dab1_0      4123
Tbr1_0      3466
NT_0        2735
Rb1cc1_0    2534
Satb2_0     2320
Herc1_0     2165
NT_1        1979
Ank3_0      1763
Xpo7_0      1602
Cx3cl1_0    1392
Cul1_0      1297
ST_0         307
ST_1         203
Name: count, dtype: int64

In [15]:
def clean_assign(val):
    if val in {'NT_0', 'NT_1', 'ST_0', 'ST_1'}:
        return 'ctrl'
    else:
        return val.replace('_0', '').capitalize()

adata.obs['Assign_clean'] = adata.obs['Assign'].apply(clean_assign)

In [16]:
# Show the unique cell types
adata.obs['Assign_clean'].value_counts()

Assign_clean
Bcl11b    5367
ctrl      5224
Trio      4274
Dab1      4123
Tbr1      3466
Rb1cc1    2534
Satb2     2320
Herc1     2165
Ank3      1763
Xpo7      1602
Cx3cl1    1392
Cul1      1297
Name: count, dtype: int64

In [17]:
adata.obs['condition'] = adata.obs['Assign_clean'].apply(
    lambda x: x if x == 'ctrl' else f"{x.split('+')[0].upper()}+ctrl"
)

In [18]:
adata.obs['condition'].value_counts()

condition
BCL11B+ctrl    5367
ctrl           5224
TRIO+ctrl      4274
DAB1+ctrl      4123
TBR1+ctrl      3466
RB1CC1+ctrl    2534
SATB2+ctrl     2320
HERC1+ctrl     2165
ANK3+ctrl      1763
XPO7+ctrl      1602
CX3CL1+ctrl    1392
CUL1+ctrl      1297
Name: count, dtype: int64

In [19]:
adata.obs.head(2)

,orig.ident,nCount_RNA,nFeature_RNA,seurat_clusters,Assign,scds,cxds,bcds,Sample,nCount_refAssay,...,predicted.subclass,CT,mito,BioSamp,CT2,ForPlot,Remove,active_ident,Assign_clean,condition
batch1_samp1_chan1_AAACCTGAGACAAAGG-1,batch1,6831,2953,7,Bcl11b_0,1.323674,28996.749458,0.880423,batch1_samp1_chan1,6831,...,L5 PT CTX,L6 CT CTX,0.029278,batch1_samp1,030 L6 CT CTX Glut,batch1_samp1_Bcl11b_0,False,7,Bcl11b,BCL11B+ctrl
batch1_samp1_chan1_AAACCTGAGGCCGAAT-1,batch1,6016,2767,21,Bcl11b_0,0.487555,9829.195327,0.337222,batch1_samp1_chan1,6016,...,L6b CTX,L6b CTX,0.016622,batch1_samp1,029 L6b CTX Glut,batch1_samp1_Bcl11b_0,False,21,Bcl11b,BCL11B+ctrl


In [20]:
adata.obs['cell_type'] = 'Neuron'

In [23]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata,n_top_genes=5000, subset=True)

In [25]:
adata

AnnData object with n_obs × n_vars = 35527 × 5000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'seurat_clusters', 'Assign', 'scds', 'cxds', 'bcds', 'Sample', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'CT', 'mito', 'BioSamp', 'CT2', 'ForPlot', 'Remove', 'active_ident', 'Assign_clean', 'condition', 'cell_type', 'dose_val', 'control', 'condition_name'
    var: 'variable_gene', 'gene_name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg', 'rank_genes_groups_cov_all', 'top_non_dropout_de_20', 'non_dropout_gene_idx', 'non_zeros_gene_idx', 'top_non_zero_de_20'

In [28]:
import sys
sys.path.append('../')

from gears import PertData

pert_data = PertData('./data') # specific saved folder
pert_data.new_data_process(dataset_name = 'boli', adata = adata) # specific dataset name and adata object
pert_data.load(data_path = './data/boli') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split = 'simulation_single', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
Found local copy...
Creating pyg object for each cell in the data...
Creating dataset file...
 17%|███████████▊                                                           | 2/12 [00:28<02:08, 12.82s/it]

TRIO+ctrl


 58%|█████████████████████████████████████████▍                             | 7/12 [01:14<00:42,  8.41s/it]

DAB1+ctrl


 75%|█████████████████████████████████████████████████████▎                 | 9/12 [01:35<00:27,  9.07s/it]

CX3CL1+ctrl


100%|██████████████████████████████████████████████████████████████████████| 12/12 [01:59<00:00,  9.96s/it]
Done!
Saving new dataset pyg object at ./data/boli/data_pyg/cell_graphs.pkl
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['TRIO+ctrl' 'DAB1+ctrl' 'CX3CL1+ctrl']
Local copy of pyg dataset is detected. Loading...
Done!
Creating new splits....


ValueError: too many values to unpack (expected 2)

In [1]:
import os
os.chdir("/gpfs/home/asun/jin_lab/get/GEARS/demo/")
print(os.getcwd())

import sys
sys.path.append(os.path.abspath('..'))

from gears import PertData

/gpfs/group/jin/asun/get/GEARS/demo


In [2]:
pert_data = PertData('./data') # specific saved folder
#pert_data.new_data_process(dataset_name = 'boli', adata = adata) # specific dataset name and adata object
pert_data.load(data_path = './data/boli') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split = 'simulation_single', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['TRIO+ctrl' 'DAB1+ctrl' 'CX3CL1+ctrl']
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Done!
Creating dataloaders....
Done!


here1
dict_keys(['test', 'train', 'val'])
simulation_single


# checking dense vs sparse


In [19]:
boli_adata = sc.read_h5ad('/gpfs/home/asun/jin_lab/get/GEARS/demo/data/boli/perturb_processed.h5ad')


In [21]:
type(boli_adata.X)               # Is it <class 'numpy.ndarray'> or <class 'scipy.sparse.csr.csr_matrix'>?
adata.X[:5]                 # Try printing a few rows — is it sparse-looking?


scipy.sparse._csr.csr_matrix

In [20]:
norman_adata = sc.read_h5ad('/gpfs/home/asun/jin_lab/get/GEARS/demo/data/norman/perturb_processed.h5ad')


In [22]:
type(norman_adata.X)               # Is it <class 'numpy.ndarray'> or <class 'scipy.sparse.csr.csr_matrix'>?


scipy.sparse._csr.csr_matrix